# Project 5. H+, or how to build a perfect human.

Lab notebook by Nikita Vyatkin and Asya Bukreeva

In [1]:
import pandas as pd

In [2]:
def show_file_content(path, lines=10, headers_only=False):
    if headers_only:
        ! mkdir -p temp
        ! touch temp/show_file_content.tmp
        ! grep '>' {path} > temp/show_file_content.tmp
        path = "temp/show_file_content.tmp"
    
    ! head -{lines} {path}
    print("...")
    ! tail -{lines} {path}
    
    if headers_only:
        ! rm {path}


def count_seqs(path):
    res = ! grep '>' {path} | wc -l
    print(f"Total: {res[0]} sequencies")

def find_snps_by_rsid(file_name, rsids):
    for rsid in rsids:
        ! grep {rsid} {file_name}

## Data source

We will work with raw 23andMe data from the “GitHub Guy” (https://github.com/msporny/dna): released under Creative Commons Public Domain License.

In [3]:
! ls -lh raw_data

total 39M
-rw-rw-r-- 1 nikitos nikitos 24M фев 12 15:48 ManuSporny-genome.txt
-rw-rw-r-- 1 nikitos nikitos 15M мар 24  2020 MikeRaiko-genome.txt


In [15]:
show_file_content("raw_data/ManuSporny-genome.txt", lines=20)

# This data file generated by 23andMe at: Sat Feb 12 07:54:57 2011
#
# Below is a text version of your data. Fields are TAB-separated
# Each line corresponds to a single SNP.  For each SNP, we provide its identifier 
# (an rsid or an internal id), its location on the reference human genome, and the 
# genotype call oriented with respect to the plus strand on the human reference 
# sequence.     We are using reference human assembly build 36.  Note that it is possible 
# that data downloaded at different times may be different due to ongoing improvements 
# in our ability to call genotypes. More information about these changes can be found at:
# https://www.23andme.com/you/download/revisions/
# 
# More information on reference human assembly build 36:
# http://www.ncbi.nlm.nih.gov/projects/mapview/map_search.cgi?taxid=9606&build=36
#
# rsid	chromosome	position	genotype
rs4477212	1	72017	AA
rs3094315	1	742429	AA
rs3131972	1	742584	GG
rs12124819	1	766409	AA
rs11240777	1	788822	AG
...
i300

## File conversion

For analysis we will need to convert 23andMe's raw data into standard vcf format. We will use [plink](https://www.cog-genomics.org/plink/), a program widely used in population genetics. One can download plink [here](https://www.cog-genomics.org/plink/)

In [16]:
! plink

PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3

  plink <input flag(s)...> [command flag(s)...] [other flag(s)...]
  plink --help [flag name(s)...]

Commands include --make-bed, --recode, --flip-scan, --merge-list,
--write-snplist, --list-duplicate-vars, --freqx, --missing, --test-mishap,
--hardy, --mendel, --ibc, --impute-sex, --indep-pairphase, --r2, --show-tags,
--blocks, --distance, --genome, --homozyg, --make-rel, --make-grm-gz,
--rel-cutoff, --cluster, --pca, --neighbour, --ibs-test, --regress-distance,
--model, --bd, --gxe, --logistic, --dosage, --lasso, --test-missing,
--make-perm-pheno, --tdt, --qfam, --annotate, --clump, --gene-report,
--meta-analysis, --epistasis, --fast-epistasis, and --score.

"plink --help | more" describes all functions (warning: long).


In [17]:
! plink --help

PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3

In the command line flag definitions that follow,
  * <angle brackets> denote a required parameter, where the text between the
    angle brackets describes its nature.
  * ['square brackets + single-quotes'] denotes an optional modifier.  Use the
    EXACT text in the quotes.
  * [{bar|separated|braced|bracketed|values}] denotes a collection of mutually
    exclusive optional modifiers (again, the exact text must be used).  When
    there are no outer square brackets, one of the choices must be selected.
  * ['quoted_text='<description of value>] denotes an optional modifier that
    must begin with the quoted text, and be followed by a value with no
    whitespace in between.  '|' may also be used here to indicate mutually
    exclusive options.
  * [square brackets without quotes or braces] denote an optional parameter,
    whe

In [18]:
! mkdir data

In [19]:
! plink --23file raw_data/ManuSporny-genome.txt --recode vcf --out data/snps_clean --output-chr MT --snps-only just-acgt

PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to data/snps_clean.log.
Options in effect:
  --23file raw_data/ManuSporny-genome.txt
  --out data/snps_clean
  --output-chr MT
  --recode vcf
  --snps-only just-acgt

15778 MB RAM detected; reserving 7889 MB for main workspace.
--23file: data/snps_clean-temporary.bed + data/snps_clean-temporary.bim +
data/snps_clean-temporary.fam written.
1090 variants with indel calls present.  '--snps-only no-DI' or
--list-23-indels may be useful here.
Inferred sex: male.
965887 out of 966977 variants loaded from .bim file.
1 person (1 male, 0 females) loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1 founder and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667

In [9]:
! plink --23file raw_data/MikeRaiko-genome.txt --recode vcf --out data/snps_clean_MikeRaiko --output-chr MT --snps-only just-acgt

PLINK v1.90b6.21 64-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to data/snps_clean_MikeRaiko.log.
Options in effect:
  --23file raw_data/MikeRaiko-genome.txt
  --out data/snps_clean_MikeRaiko
  --output-chr MT
  --recode vcf
  --snps-only just-acgt

15778 MB RAM detected; reserving 7889 MB for main workspace.
--23file: data/snps_clean_MikeRaiko-temporary.bed +
data/snps_clean_MikeRaiko-temporary.bim +
data/snps_clean_MikeRaiko-temporary.fam written.
15125 variants with indel calls present.  '--snps-only no-DI' or
--list-23-indels may be useful here.
Inferred sex: male.
595401 out of 610526 variants loaded from .bim file.
1 person (1 male, 0 females) loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1 founder and 0 nonfounders present.
Calculating allele frequencies... 101112131415161718192021222324252627282930313233343536373839404142

## Origins, haplogroups

Establish probable ethnicity of given subject by identifying maternal (mtDNA) and paternal (Y chromosome) haplogroups.

https://dna.jameslick.com/mthap/ - shows all SNPs that distinguish the haplogroup, and takes 23andMe input.

### Mike Raiko:


mthap version 0.19b (2015-05-11); haplogroup data version PhyloTree Build 17 (2016-02-18) +mods
raw data source MikeRaiko-genome.txt (14MB)

Found 3270 markers at 3268 positions covering 19.7% of mtDNA.

Markers found (shown as differences to rCRS):

HVR2: 152C 263G
CR: 750G 1438G 4769G 8860G
HVR1:

IMPORTANT NOTE: The above marker list is almost certainly incomplete due to limitations of genotyping technology and is not comparable to mtDNA sequencing results. It should not be used with services or tools that expect sequencing results, such as mitosearch.

Best mtDNA Haplogroup Matches:

1) H(T152C)

Defining Markers for haplogroup H(T152C):
HVR2: 152C 263G
CR: 750G 1438G 4769G 8860G 15326G
HVR1:

Marker path from rCRS to haplogroup H(T152C):
H2a2a1(rCRS) ⇨ 263G ⇨ H2a2a ⇨ 8860G 15326G ⇨ H2a2 ⇨ 750G ⇨ H2a ⇨ 4769G ⇨ H2 ⇨ 1438G ⇨ H ⇨ 152C ⇨ H(T152C)

Imperfect Match. Your results contained differences with this haplogroup:
Matches(6): 152C 263G 750G 1438G 4769G 8860G
Untested(1): 15326

2) H1(T152C)

Defining Markers for haplogroup H1(T152C):
HVR2: 152C 263G
CR: 750G 1438G 3010A 4769G 8860G 15326G
HVR1:

Marker path from rCRS to haplogroup H1(T152C):
H2a2a1(rCRS) ⇨ 263G ⇨ H2a2a ⇨ 8860G 15326G ⇨ H2a2 ⇨ 750G ⇨ H2a ⇨ 4769G ⇨ H2 ⇨ 1438G ⇨ H ⇨ 3010A ⇨ H1 ⇨ 152C ⇨ H1(T152C)

Imperfect Match. Your results contained differences with this haplogroup:
Matches(6): 152C 263G 750G 1438G 4769G 8860G
No-Calls(1): 3010A
Untested(1): 15326

3) H

Defining Markers for haplogroup H:
HVR2: 263G
CR: 750G 1438G 4769G 8860G 15326G
HVR1:

Marker path from rCRS to haplogroup H (plus extra markers):
H2a2a1(rCRS) ⇨ 263G ⇨ H2a2a ⇨ 8860G 15326G ⇨ H2a2 ⇨ 750G ⇨ H2a ⇨ 4769G ⇨ H2 ⇨ 1438G ⇨ H ⇨ 152C

Imperfect Match. Your results contained differences with this haplogroup:
Matches(5): 263G 750G 1438G 4769G 8860G
Extras(1): 152C
Untested(1): 15326

3) H9

Defining Markers for haplogroup H9:
HVR2: 152C 263G
CR: 750G 1438G 4769G 8860G 13020C 15326G
HVR1:

Marker path from rCRS to haplogroup H9:
H2a2a1(rCRS) ⇨ 263G ⇨ H2a2a ⇨ 8860G 15326G ⇨ H2a2 ⇨ 750G ⇨ H2a ⇨ 4769G ⇨ H2 ⇨ 1438G ⇨ H ⇨ 152C ⇨ H(T152C) ⇨ 13020C ⇨ H9

Imperfect Match. Your results contained differences with this haplogroup:
Matches(6): 152C 263G 750G 1438G 4769G 8860G
Mismatches(1): 13020T
Untested(1): 15326

3) H46

Defining Markers for haplogroup H46:
HVR2: 152C 263G
CR: 750G 1438G 2772T 4769G 8860G 15326G
HVR1:

Marker path from rCRS to haplogroup H46:
H2a2a1(rCRS) ⇨ 263G ⇨ H2a2a ⇨ 8860G 15326G ⇨ H2a2 ⇨ 750G ⇨ H2a ⇨ 4769G ⇨ H2 ⇨ 1438G ⇨ H ⇨ 152C ⇨ H(T152C) ⇨ 2772T ⇨ H46

Imperfect Match. Your results contained differences with this haplogroup:
Matches(6): 152C 263G 750G 1438G 4769G 8860G
Mismatches(1): 2772C
Untested(1): 15326

3) H3(T152C)

Defining Markers for haplogroup H3(T152C):
HVR2: 152C 263G
CR: 750G 1438G 4769G 6776C 8860G 15326G
HVR1:

Marker path from rCRS to haplogroup H3(T152C):
H2a2a1(rCRS) ⇨ 263G ⇨ H2a2a ⇨ 8860G 15326G ⇨ H2a2 ⇨ 750G ⇨ H2a ⇨ 4769G ⇨ H2 ⇨ 1438G ⇨ H ⇨ 6776C ⇨ H3 ⇨ 152C ⇨ H3(T152C)

Imperfect Match. Your results contained differences with this haplogroup:
Matches(6): 152C 263G 750G 1438G 4769G 8860G
Mismatches(1): 6776T
Untested(1): 15326

3) H52

Defining Markers for haplogroup H52:
HVR2: 152C 263G
CR: 750G 1438G 4769G 8860G 14220G 15326G
HVR1:

Marker path from rCRS to haplogroup H52:
H2a2a1(rCRS) ⇨ 263G ⇨ H2a2a ⇨ 8860G 15326G ⇨ H2a2 ⇨ 750G ⇨ H2a ⇨ 4769G ⇨ H2 ⇨ 1438G ⇨ H ⇨ 152C ⇨ H(T152C) ⇨ 14220G ⇨ H52

Imperfect Match. Your results contained differences with this haplogroup:
Matches(6): 152C 263G 750G 1438G 4769G 8860G
Mismatches(1): 14220A
Untested(1): 15326

3) H16(T152C)

Defining Markers for haplogroup H16(T152C):
HVR2: 152C 263G
CR: 750G 1438G 4769G 8860G 10394T 15326G
HVR1:

Marker path from rCRS to haplogroup H16(T152C):
H2a2a1(rCRS) ⇨ 263G ⇨ H2a2a ⇨ 8860G 15326G ⇨ H2a2 ⇨ 750G ⇨ H2a ⇨ 4769G ⇨ H2 ⇨ 1438G ⇨ H ⇨ 10394T ⇨ H16 ⇨ 152C ⇨ H16(T152C)

Imperfect Match. Your results contained differences with this haplogroup:
Matches(6): 152C 263G 750G 1438G 4769G 8860G
Mismatches(1): 10394C
Untested(1): 15326

3) H69

Defining Markers for haplogroup H69:
HVR2: 152C 263G
CR: 750G 1438G 4646C 4769G 8860G 15326G
HVR1:

Marker path from rCRS to haplogroup H69:
H2a2a1(rCRS) ⇨ 263G ⇨ H2a2a ⇨ 8860G 15326G ⇨ H2a2 ⇨ 750G ⇨ H2a ⇨ 4769G ⇨ H2 ⇨ 1438G ⇨ H ⇨ 152C ⇨ H(T152C) ⇨ 4646C ⇨ H69

Imperfect Match. Your results contained differences with this haplogroup:
Matches(6): 152C 263G 750G 1438G 4769G 8860G
Mismatches(1): 4646T
Untested(1): 15326 

## Annotation - sex and eye colour

In [4]:
eye_skin_colour_rsids = ["rs12913832",
                         "rs12203592",
                         "rs16891982",
                         "rs6119471",
                         "rs12896399",
                         "rs1426654",
                         "rs1545397",
                         "rs885479",
                        ]

find_snps_by_rsid("raw_data/MikeRaiko-genome.txt", eye_skin_colour_rsids)
print("-----")
find_snps_by_rsid("data/snps_clean_MikeRaiko.vcf", eye_skin_colour_rsids)

rs12913832	15	28365618	AG
rs12203592	6	396321	CT
rs16891982	5	33951693	CG
rs12896399	14	92773663	GG
rs1426654	15	48426484	AA
rs885479	16	89986154	GG
-----
15	28365618	rs12913832	A	G	.	.	PR	GT	0/1
6	396321	rs12203592	C	T	.	.	PR	GT	0/1
5	33951693	rs16891982	C	G	.	.	PR	GT	0/1
14	92773663	rs12896399	G	.	.	.	PR	GT	0/0
15	48426484	rs1426654	A	.	.	.	PR	GT	0/0
16	89986154	rs885479	G	.	.	.	PR	GT	0/0


Eye colour: Brown

Skin colour: light or medium (not dark)

In [8]:
find_snps_by_rsid("raw_data/ManuSporny-genome.txt", eye_skin_colour_rsids)

rs12913832	15	26039213	AG
rs12203592	6	341321	CC
rs16891982	5	33987450	CG
rs12896399	14	91843416	GT
rs1426654	15	46213776	AG
rs885479	16	88513655	GG


## Annotation of all SNPs, selection of clinically relevant ones.


In [5]:
! awk '($32!="-") ' VEP_results/MikeRaiko/G9fJSvE7sdcvGHwt.txt | grep risk_factor | cut -f 1-3 | sort | uniq

i3000469	2:138759649-138759649	T
i6007787	2:234183368-234183368	G
i6058143	1:161479745-161479745	G
i6059141	8:133909974-133909974	G
rs1024611	17:32579788-32579788	G
rs1049296	3:133494354-133494354	T
rs10757274	9:22096055-22096055	G
rs1169288	12:121416650-121416650	C
rs12150220	17:5485367-5485367	T
rs13266634	8:118184783-118184783	T
rs1801197	7:93055753-93055753	G
rs1801274	1:161479745-161479745	G
rs1801275	16:27374400-27374400	G
rs1801394	5:7870973-7870973	G
rs1801968	9:132580901-132580901	G
rs2004640	7:128578301-128578301	T
rs2073658	1:161010762-161010762	T
rs2184026	9:101304348-101304348	T
rs2239704	6:31540141-31540141	C
rs2241880	2:234183368-234183368	G
rs2281845	1:201081943-201081943	T
rs231775	2:204732714-204732714	G
rs4402960	3:185511687-185511687	T
rs4880	6:160113872-160113872	G
rs4961	4:2906707-2906707	T
rs4977574	9:22098574-22098574	G
rs5174	1:53712727-53712727	T
rs5186	3:148459988-148459988	C
rs61747071	16:53720436-53720436	T
rs6265	11:27679916-27679916	T
rs6280	3:113890815-1

i3000469	2:138759649-138759649	T : Inherited susceptibility to asthma
i6007787	2:234183368-234183368	G : Inflammatory bowel disease 10, susceptibility to
i6058143	1:161479745-161479745	G : Lupus nephritis, susceptibility to, Pseudomonas aeruginosa, susceptibility to chronic infection by, in cystic fibrosis, Malaria, severe, susceptibility to
i6059141	8:133909974-133909974	G : Autoimmune thyroid disease, susceptibility to, 3, Iodotyrosyl coupling defect
rs1024611	17:32579788-32579788	G : Coronary artery disease, development of, in HIV, Mycobacterium tuberculosis, susceptibility to, Spina bifida, susceptibility to, Coronary artery disease, modifier of
rs1049296	3:133494354-133494354	T : Alzheimer disease, susceptibility to, Atransferrinemia, Transferrin variant c1/c2, not provided
rs10757274	9:22096055-22096055	G
rs1169288	12:121416650-121416650	C
rs12150220	17:5485367-5485367	T
rs13266634	8:118184783-118184783	T
rs1801197	7:93055753-93055753	G
rs1801274	1:161479745-161479745	G
rs1801275	16:27374400-27374400	G
rs1801394	5:7870973-7870973	G
rs1801968	9:132580901-132580901	G
rs2004640	7:128578301-128578301	T
rs2073658	1:161010762-161010762	T
rs2184026	9:101304348-101304348	T
rs2239704	6:31540141-31540141	C
rs2241880	2:234183368-234183368	G
rs2281845	1:201081943-201081943	T
rs231775	2:204732714-204732714	G
rs4402960	3:185511687-185511687	T
rs4880	6:160113872-160113872	G
rs4961	4:2906707-2906707	T
rs4977574	9:22098574-22098574	G
rs5174	1:53712727-53712727	T
rs5186	3:148459988-148459988	C
rs61747071	16:53720436-53720436	T
rs6265	11:27679916-27679916	T
rs6280	3:113890815-113890815	T
rs6504649	17:48437456-48437456	G
rs699	1:230845794-230845794	G
rs763110	1:172627498-172627498	T
rs7794745	7:146489606-146489606	T
rs909253	6:31540313-31540313	G

In [7]:
vcf_file = "VEP_results/MikeRaiko/G9fJSvE7sdcvGHwt.vcf"

In [9]:
# Create a dictionary to store the data
variants = {}

# Read in the VCF file line by line
with open(vcf_file, 'r') as f:
    for line in f:
        # Skip the header lines
        if line.startswith('#'):
            continue
        # Split the line into columns
        cols = line.strip().split('\t')
        # Get the variant information
        chrom = cols[0]
        pos = cols[1]
        ref = cols[3]
        alt = cols[4]
        # Get the effect information
        info = cols[7]
        # Check if the variant increases the risk of disease
        if 'risk_factor' in info:
            # Store the variant information in the dictionary
            variants[(chrom, pos, ref, alt)] = info

# Print out the variants that increase the risk of disease
for variant in variants:
    chrom, pos, ref, alt = variant
    info = variants[variant]
    print('Variant: {0}:{1} {2}>{3} - {4}'.format(chrom, pos, ref, alt, info))

Variant: 1:53712727 C>T - PR;CSQ=T|missense_variant&splice_region_variant|MODERATE|LRP8|ENSG00000157193|Transcript|ENST00000306052.6|protein_coding|19/19||||2957|2855|952|R/Q|cGa/cAa|rs5174&CM074332&COSV60098463||-1||HGNC|6700|||||deleterious_low_confidence(0.01)|probably_damaging(0.978)|0.1444|risk_factor|0&0&1|1&1&1|16642433&21738487&31397093&22938532&17847002&15986317&18592168&21316997&22404453&22889673&26637325&29032149&24867879&18245017&30622122&30643258&26902204&19439088&12399018|||||,T|missense_variant&splice_region_variant|MODERATE|LRP8|ENSG00000157193|Transcript|ENST00000347547.2|protein_coding|17/17||||2345|2345|782|R/Q|cGa/cAa|rs5174&CM074332&COSV60098463||-1||HGNC|6700|||||deleterious_low_confidence(0.01)|probably_damaging(0.99)|0.1444|risk_factor|0&0&1|1&1&1|16642433&21738487&31397093&22938532&17847002&15986317&18592168&21316997&22404453&22889673&26637325&29032149&24867879&18245017&30622122&30643258&26902204&19439088&12399018|||||,T|missense_variant&splice_region_variant|M